## SeqNeo
- Stage: Cambrian
- Version: Pomoria

#### Related Commands
- tensorboard --logdir runs
- nohup jupyter lab > jupyter.log &

In [1]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [2]:
import os
import itertools
import copy
import random
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: True


In [3]:
import tyrell.spec as S
from tyrell.decider import Example

# Morpheus Version
from MorpheusInterpreter import *
from ProgramSpace import *

In [4]:
torch.__version__

'1.0.0'

In [5]:
class SeqNeo(nn.Module):
    def __init__(self, p_config=None):
        super(SeqNeo, self).__init__()
        self.config = p_config
        
        self.fn_embedding = nn.Embedding(
            self.config["fn_vocab_size"],
            self.config["fn_dim"],
            padding_idx=0,
        )
        
        self.encoder1 = nn.Linear(
            self.config["encoder"]["input_dim"],
            2048,
        )
        self.encoder2 = nn.Linear(
            2048,
            self.config["encoder"]["output_dim"],
        )
        
        self.decoder = nn.GRUCell(
            self.config["decoder"]["input_dim"],
            self.config["decoder"]["hidden_dim"],
            bias=True,
        )
        self.classifier = nn.Linear(
            self.config["decoder"]["hidden_dim"],
            self.config["decoder"]["output_dim"],
        )
        
    def decode(self, pin_exp, pout_exp, p_hidden, pfn_context):
        tmp_exp = self.encode(pin_exp, pout_exp) # (B=1, encoder_output_dim)
        tmp_fn = self.fn_embedding(pfn_context) # (B=1, fn_dim)
        tmp_input = torch.cat([tmp_exp,tmp_fn],dim=1)
        
        if p_hidden is None:
            # very first
            tmp_hidden = self.decoder(tmp_input)
        else:
            # not the very first
            tmp_hidden = self.decoder(tmp_input, p_hidden)
        # (B=1, decoder_hidden_dim)
        
        tmp_output = torch.log_softmax(
            self.classifier(tmp_hidden),
            dim=1
        ) # (B=1, decoder_output_dim)
        
        # first hidden, then output
        return (tmp_hidden, tmp_output)
        
        
    def encode(self, pin_exp, pout_exp):
        # pin_exp/pout_exp: (B=1, abs_dim)
        tmp_exp = torch.cat([pin_exp,pout_exp],dim=1) # (B=1, encoder_input_dim=abs_dim*2)
        tmp_out1 = F.relu(self.encoder1(tmp_exp))
        tmp_out2 = F.relu(self.encoder2(tmp_out1))
        return tmp_out2 # (B=1, encoder_output_dim)
    

In [6]:
# replace certain node id with certain value
def modify_shell(p_shell, p_id_from, p_id_to):
    d_prod = p_shell[0]
    d_rhs = p_shell[1]
    ld_rhs = [p_id_to if d_rhs[i]==p_id_from else d_rhs[i]
             for i in range(len(d_rhs))]
    return (d_prod, tuple(ld_rhs))

def SeqTrain(p_config, p_sourceps, p_model, p_dataset, p_optim):
    print("# Start SeqTrain...")
    
    for d_epoch in range(p_config["train"]["n_epoch"]):
        epoch_loss_list = []
        for d_episode in range(p_config["train"]["n_episode"]):
            print("\r# EP:{}/{}, loss:{:.2f}".format(
                d_epoch, d_episode,
                sum(epoch_loss_list)/len(epoch_loss_list) if len(epoch_loss_list)>0 else -1,
            ),end="")
            p_model.train()
            
            if isinstance(p_dataset,list):
                # ====== option 1: sample from dataset ====== #
                eid = random.choice(range(len(p_dataset)))
                data_prog, data_str_example = p_dataset[eid]
                data_example = Example(
                    input=[p_sourceps.interpreter.load_data_into_var(p) for p in data_str_example.input],
                    output=p_sourceps.interpreter.load_data_into_var(data_str_example.output),
                )
                ps_solution = ProgramSpace(
                    p_sourceps.spec, p_sourceps.interpreter, data_example.input, data_example.output,
                )
                ps_solution.init_by_prog(data_prog)
            else:
                # ====== option 2: sample from generator ====== #
                ps_solution = p_dataset.get_new_chain_program(
                    p_config["train"]["n_size"] + 1 # depth=size+1
                )

            # solution self-check
            if ps_solution.check_eq() is None:
                continue
                print("ERROR, SOLUTION NOT CONSISTENT!")

            selected_neurons = []

            # initialize a new Program Space
            ps_current = ProgramSpace(
                p_sourceps.spec, p_sourceps.interpreter, ps_solution.inputs, ps_solution.output,
            )
            # then initialize a shell template
            tmp_shell_list = ps_current.get_neighboring_shells()
            tmp_node_to_replace = ps_current.node_dict["ParamNode"][0] # for chain only
            # replace the Param Node id in shells with -1 to make them templates
            template_list = [
                modify_shell(tmp_shell_list[i],tmp_node_to_replace,-1)
                for i in range(len(tmp_shell_list))
            ]

            hidden_current = None # hidden state, initialized to None
            fn_context = 0
            for d_step in range(len(ps_solution.shells)):

                # ### assume chain execution, so only 1 possible returns
                # ### at d_step=0, this should be input[0]
                id_current = ps_current.get_strict_frontiers()[0]
                var_current = ps_current.node_list[id_current].ps_data # need the real var name in r env
                var_output = ps_current.output

                map_current = p_sourceps.interpreter.camb_get_ventogyrus(var_current)
                map_output = p_sourceps.interpreter.camb_get_ventogyrus(var_output)

                # make current shell list
                current_shell_list = [
                    modify_shell(template_list[i],-1,id_current)
                    for i in range(len(template_list))
                ]
                current_shell_list = [None] + current_shell_list # None for <SOS> in nn embedding

                # wrap in B=1
                if use_cuda:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float)).cuda()
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float)).cuda()
                    td_context = Variable(torch.tensor([fn_context],dtype=torch.long)).cuda()
                else:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float))
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float))
                    td_context = Variable(torch.tensor([fn_context],dtype=torch.long))

                # (B=1, fn_vocab_size)
                hidden_current, td_pred = p_model.decode(td_current, td_output, hidden_current, td_context)
                # supervised, assign selection directly
                tmp_id = current_shell_list.index(ps_solution.shells[d_step])
                fn_context = tmp_id

                # update ps_current
                update_status = ps_current.add_neighboring_shell(
                    current_shell_list[tmp_id]
                )
                
                if not update_status:
                    # something wrong
                    raise Exception("Can't even update PS under supervision?")
                
                selected_neurons.append(td_pred[0,tmp_id])

            # <END_FOR_STEP>
            if ps_current.check_eq() is None:
                # something wrong, should've been solved under supervision
                raise Exception("Can't even solve under supervision?")

            episode_loss_list = []
            for i in range(len(selected_neurons)):
                episode_loss_list.append(
                    (+1.0)*(-selected_neurons[i])
                )
            episode_loss = sum(episode_loss_list)
            epoch_loss_list.append(episode_loss)
            p_optim.zero_grad()
            episode_loss.backward()
            p_optim.step()

                
        # <END_FOR_EPISODE>  
        print()
        if d_epoch % 10 == 0:
            torch.save(p_model.state_dict(),"./saved_models/0811CambAgent_Thomas_Size3_ep{}.pt".format(d_epoch))
    # <END_FOR_EPOCH>
    

In [7]:
def SeqBeam(p_config, p_sourceps, p_model, p_dataset):
    print("# Start Beam Search...")
    
    for d_episode in range(p_config["train"]["n_episode"]):
        p_model.eval()

        if isinstance(p_dataset,list):
            # ====== option 1: sample from dataset ====== #
            eid = random.choice(range(len(p_dataset)))
            data_prog, data_str_example = p_dataset[eid]
            data_example = Example(
                input=[p_sourceps.interpreter.load_data_into_var(p) for p in data_str_example.input],
                output=p_sourceps.interpreter.load_data_into_var(data_str_example.output),
            )
            ps_solution = ProgramSpace(
                p_sourceps.spec, p_sourceps.interpreter, data_example.input, data_example.output,
            )
            ps_solution.init_by_prog(data_prog)
        else:
            # ====== option 2: sample from generator ====== #
            ps_solution = p_dataset.get_new_chain_program(
                p_config["train"]["n_size"] + 1 # depth=size+1
            )

        # solution self-check
        if ps_solution.check_eq() is None:
            continue
            print("ERROR, SOLUTION NOT CONSISTENT!")
        else:
            print("==== Problem ====")
            print(str(ps_solution.node_list[-1]))

        selected_neurons = []

        # initialize a new Program Space
        ps_current = ProgramSpace(
            p_sourceps.spec, p_sourceps.interpreter, ps_solution.inputs, ps_solution.output,
        )
        # then initialize a shell template
        tmp_shell_list = ps_current.get_neighboring_shells()
        tmp_node_to_replace = ps_current.node_dict["ParamNode"][0] # for chain only
        # replace the Param Node id in shells with -1 to make them templates
        template_list = [
            modify_shell(tmp_shell_list[i],tmp_node_to_replace,-1)
            for i in range(len(tmp_shell_list))
        ]
        
        beam_list = [
            # (score, hidden, fn_context, ProgramSpace)
            (0.0, None, 0, ps_current.make_copy())
        ]

        for d_step in range(len(ps_solution.shells)):
            sprout_list = []
            for d_bid in range(len(beam_list)):
                dd_score, dd_hidden, dd_context, dd_ps = beam_list[d_bid]
                ps_current = dd_ps
                
                # ### assume chain execution, so only 1 possible returns
                # ### at d_step=0, this should be input[0]
                id_current = ps_current.get_strict_frontiers()[0]
                var_current = ps_current.node_list[id_current].ps_data # need the real var name in r env
                var_output = ps_current.output

                map_current = p_sourceps.interpreter.camb_get_ventogyrus(var_current)
                map_output = p_sourceps.interpreter.camb_get_ventogyrus(var_output)

                # make current shell list
                current_shell_list = [
                    modify_shell(template_list[i],-1,id_current)
                    for i in range(len(template_list))
                ]
                current_shell_list = [None] + current_shell_list # None for <SOS> in nn embedding

                # wrap in B=1
                if use_cuda:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float)).cuda()
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float)).cuda()
                    td_context = Variable(torch.tensor([dd_context],dtype=torch.long)).cuda()
                else:
                    td_current = Variable(torch.tensor([map_current],dtype=torch.float))
                    td_output = Variable(torch.tensor([map_output],dtype=torch.float))
                    td_context = Variable(torch.tensor([dd_context],dtype=torch.long))

                # (B=1, fn_vocab_size)
                hidden_current, td_pred = p_model.decode(td_current, td_output, dd_hidden, td_context)
                
                # start sprouting
                np_pred = td_pred.data.cpu().numpy().flatten()
                as_pred = np.argsort(np_pred)[::-1]
                for i in range(p_config["test"]["sprout_size"]):
                    ppps = ps_current.make_copy()
                    tmp_id = as_pred[i]
                    fn_context = tmp_id
                    # try to update
                    update_status = ppps.add_neighboring_shell(
                        current_shell_list[tmp_id]
                    )
                    if update_status:
                        # success, put into sprout list
                        sprout_list.append(
                            (
                                dd_score+np_pred[tmp_id],
                                hidden_current,
                                fn_context,
                                ppps,
                            )
                        )
                    # else: do nothing
                # <END_FOR_SPROUT>
            # <END_FOR_BEAM>
            sorted_sprout_list = sorted(
                sprout_list,
                key=lambda x:x[0],
                reverse=True,
            )
            if len(sorted_sprout_list)>p_config["test"]["beam_size"]:
                beam_list = sorted_sprout_list[:p_config["test"]["beam_size"]]
            else:
                beam_list = sorted_sprout_list
        # <END_FOR_STEP>
        
        # then print and seek for correct solutions
        print("==== beam search ====")
        for i in range(len(beam_list)):
            dd_score, dd_hidden, dd_context, dd_ps = beam_list[i]
            print("#{}{}, {:.4f}, {}".format(
                "(solved)" if dd_ps.check_eq() is not None else "",
                i+1,
                dd_score,
                str(dd_ps.node_list[-1])
            ))
        
        input("==== Press to enter next problem ====")

    # <END_FOR_EPISODE>
    

In [8]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/camb6.tyrell')
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
)
# dumb variable to help infer the shells
m_ps = ProgramSpace(
    m_spec, m_interpreter, [None], None,
)

In [9]:
m_config = {
    "abs_dim": 15*7+1,
    "fn_dim": 128,
    "fn_vocab_size": len(m_ps.get_neighboring_shells())+1, # with one more <SOS>
    "encoder":{
        "input_dim": None,
        "output_dim": 1024, # hidden state
    },
    "decoder":{
        "input_dim": None,
        "hidden_dim": None,
        "output_dim": None,
    },
    "train":{
        "n_epoch":500, # #epoches in total
        "n_episode": 100, # #problems per epoch
        "n_size": 3,
    },
    "test":{
        "beam_size": 40,
        "sprout_size": 20,
    },
}
m_config["encoder"]["input_dim"] = m_config["abs_dim"]*2
m_config["decoder"]["input_dim"] = \
    m_config["encoder"]["output_dim"] + \
    m_config["fn_dim"]
m_config["decoder"]["hidden_dim"] = m_config["encoder"]["output_dim"]
m_config["decoder"]["output_dim"] = m_config["fn_vocab_size"]

In [10]:
with open("./0811Size3.pkl","rb") as f:
    m_dataset = pickle.load(f)

In [11]:
seq_neo = SeqNeo(p_config=m_config)
if use_cuda:
    seq_neo = seq_neo.cuda()
optimizer = torch.optim.Adam(list(seq_neo.parameters()))

In [ ]:
# SeqTrain(m_config, m_ps, seq_neo, m_generator, optimizer)
SeqTrain(m_config, m_ps, seq_neo, m_dataset, optimizer)

# Start SeqTrain...
# EP:0/34, loss:16.46

In [22]:
SeqBeam(m_config, m_ps, seq_neo, m_dataset)

# Start Beam Search...
==== Problem ====
neg_select(separate(gather(@param0, ['1', '5']), 6), ['3'])
==== beam search ====
#1, -4.0667, neg_select(gather(separate(@param0, 3), ['1', '3']), ['1', '2'])
#2, -4.8805, unite(gather(separate(@param0, 1), ['1', '4']), 3, 4)
#(solved)3, -5.7962, neg_select(separate(gather(@param0, ['1', '5']), 6), ['3'])
#4, -6.1559, unite(gather(gather(@param0, ['1', '5']), ['1', '4']), 3, 4)
#5, -6.2042, gather(gather(separate(@param0, 1), ['1', '4']), ['1', '4'])
#6, -6.4143, neg_select(gather(separate(@param0, 3), ['3', '4']), ['1', '2'])
#7, -6.5979, neg_select(gather(separate(@param0, 3), ['3', '4']), ['3', '6'])
#8, -7.0793, summarise(group_by(separate(@param0, 1), ['1', '4']), min, 3)
#9, -7.2556, neg_select(gather(separate(@param0, 1), ['1', '5']), ['1', '2'])
#10, -7.3576, select(gather(separate(@param0, 1), ['1', '4']), ['2', '3'])


==== Press to enter next problem ==== 


==== Problem ====
neg_select(spread(gather(@param0, ['3', '4']), 4, 5), ['2', '4'])
==== beam search ====
#1, -7.5909, select(summarise(group_by(@param0, ['1']), sum, 1), ['2'])
#2, -7.7589, mutate(summarise(group_by(@param0, ['1']), sum, 1), /, 1, 2)
#3, -7.9501, mutate(summarise(group_by(@param0, ['1']), min, 1), /, 1, 2)
#4, -8.0817, neg_gather(summarise(group_by(@param0, ['3', '4']), sum, 1), ['3'])
#5, -8.1234, mutate(summarise(group_by(@param0, ['1']), mean, 1), /, 1, 2)
#6, -8.2006, neg_gather(summarise(group_by(@param0, ['3', '5']), sum, 1), ['3'])
#7, -8.2213, neg_select(summarise(group_by(@param0, ['1']), sum, 1), ['1'])
#8, -8.4102, select(summarise(group_by(@param0, ['1']), mean, 1), ['2'])
#9, -8.5070, gather(summarise(group_by(@param0, ['3', '5']), sum, 1), ['1', '2'])
#10, -8.9388, select(summarise(group_by(@param0, ['1']), min, 1), ['2'])


==== Press to enter next problem ==== 


==== Problem ====
select(mutate(gather(@param0, ['3', '4']), /, 4, 2), ['3', '5'])
==== beam search ====
#1, -6.2143, neg_select(summarise(group_by(@param0, ['1', '4']), sum, 2), ['3'])
#2, -6.6083, neg_select(summarise(group_by(@param0, ['1', '4']), min, 3), ['3'])
#3, -6.9513, neg_select(summarise(group_by(@param0, ['1', '3']), min, 3), ['1', '3'])
#4, -7.0731, neg_select(summarise(group_by(@param0, ['1', '3']), min, 3), ['1', '2'])
#5, -7.2298, neg_select(neg_gather(gather(@param0, ['2', '4']), ['2', '4']), ['2'])
#6, -7.2924, neg_select(summarise(group_by(@param0, ['1', '4']), sum, 3), ['3'])
#7, -7.4381, neg_select(summarise(group_by(@param0, ['1', '4']), max, 2), ['3'])
#8, -7.7452, neg_select(unite(gather(@param0, ['2', '4']), 3, 1), ['2', '3'])
#9, -7.9352, neg_select(summarise(group_by(@param0, ['1', '3']), min, 3), ['3'])
#10, -8.2295, neg_select(summarise(group_by(@param0, ['1', '3']), sum, 3), ['1', '2'])


...

==== Press to enter next problem ==== 


==== Problem ====
summarise(gather(mutate(@param0, /, 2, 4), ['3', '5']), min, 1)
==== beam search ====
#1, -5.4296, select(summarise(group_by(@param0, ['1']), sum, 1), ['2'])
#2, -5.4459, select(summarise(group_by(@param0, ['1', '3']), sum, 3), ['3'])
#3, -5.5176, neg_select(summarise(group_by(@param0, ['1', '3']), min, 3), ['1', '2'])
#4, -5.9697, neg_select(summarise(group_by(@param0, ['1']), sum, 1), ['1'])
#5, -6.0898, select(summarise(group_by(@param0, ['3']), sum, 1), ['2'])
#6, -6.0925, select(summarise(group_by(@param0, ['1']), mean, 1), ['2'])
#7, -6.1640, select(summarise(group_by(@param0, ['1']), sum, 3), ['2'])
#8, -6.1681, select(summarise(group_by(@param0, ['1']), mean, 2), ['2'])
#9, -6.1855, neg_select(summarise(group_by(@param0, ['1', '3']), sum, 3), ['1', '2'])
#10, -6.3301, neg_select(summarise(group_by(@param0, ['1', '3']), min, 3), ['1', '3'])


==== Press to enter next problem ==== 


==== Problem ====
gather(unite(gather(@param0, ['4', '5']), 3, 5), ['3', '4'])
==== beam search ====
#1, -4.5679, neg_select(gather(separate(@param0, 3), ['1', '3']), ['1', '2'])
#2, -5.5894, neg_select(gather(separate(@param0, 3), ['3', '4']), ['3', '6'])
#3, -6.0112, neg_select(gather(separate(@param0, 3), ['3', '4']), ['1', '2'])
#4, -6.7527, neg_select(gather(separate(@param0, 3), ['1', '5']), ['1', '2'])
#5, -6.7625, summarise(group_by(separate(@param0, 5), ['1', '5']), max, 2)
#6, -6.8747, unite(gather(separate(@param0, 3), ['3', '5']), 5, 1)
#7, -7.3128, summarise(group_by(separate(@param0, 3), ['1', '4']), sum, 2)
#8, -7.5906, neg_select(gather(separate(@param0, 3), ['3', '4']), ['3', '4'])
#9, -7.6182, neg_select(gather(separate(@param0, 3), ['4', '6']), ['1', '2'])
#10, -7.6551, neg_select(neg_gather(gather(@param0, ['1', '5']), ['1', '2']), ['2'])


KeyboardInterrupt: 